In [2]:
import datasets
import pandas as pd

data = datasets.load_dataset('cord19', 'metadata')

C:\Users\aaron\Projects\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cord19 = data['train'].to_pandas()

In [43]:
all_ids = []
for ids in cord19["url"].loc[:1000]:
    pattern = r'PMC\d{5,6}'
    matches = re.findall(pattern, ids)
    if matches != []:
        all_ids.extend(matches)

In [45]:
from Bio import Entrez
import xml.etree.ElementTree as ET
Entrez.email = "a2zdeveloper1234@gmail.com"

In [9]:
# Recursive function to extract all text from the XML tree
def extract_text(element):
    text_content = []

    if element.tag in ['ref-list', 'back', 'contrib-group', 'author-notes', 'front']:
        return ''
        
    # If the element has text, append it
    if element.text:
        text_content.append(element.text)
    
    # Recursively process all child elements
    for child in element:
        text_content.append(extract_text(child))
    
    # If the element has tail text (e.g., text following child elements), append it
    if element.tail:
        text_content.append(element.tail)
    
    return ''.join(text_content)

# Extract all the text from the XML tree
all_text = extract_text(root)


In [49]:
corpus = []
for ids in all_ids:
    try: 
        handle = Entrez.efetch(db="pmc", id=ids, rettype="full", retmode="xml")
        data = handle.read()
        handle.close()
        root = ET.fromstring(data)
        all_text = extract_text(root)
        text = re.sub(r'\[\s*\d+(?:\s*,\s*\d+)*\s*\]', "", all_text)
        corpus.append(text)
    except:
        continue

In [51]:
tokens = {}
for text in corpus:
    text_split = text.lower().split(" ")
    for word in text_split:
        new_word = " ".join(word.split()) + " </w>"
        if new_word in tokens:
            tokens[new_word] += 1
        else:
            tokens[new_word] = 1

In [52]:
tokens



{' </w>': 72492,
 'introduction </w>': 21,
 'mycoplasma </w>': 4,
 'pneumoniae </w>': 33,
 'is </w>': 1143,
 'a </w>': 1528,
 'common </w>': 46,
 'cause </w>': 35,
 'of </w>': 4707,
 'upper </w>': 19,
 'and </w>': 2748,
 'lower </w>': 53,
 'respiratory </w>': 104,
 'tract </w>': 32,
 'infections. </w>': 15,
 'it </w>': 333,
 'remains </w>': 26,
 'one </w>': 165,
 'the </w>': 5955,
 'most </w>': 162,
 'frequent </w>': 10,
 'causes </w>': 12,
 'atypical </w>': 44,
 'pneumonia </w>': 31,
 'particularly </w>': 17,
 'among </w>': 30,
 'young </w>': 7,
 'adults. </w>': 3,
 'although </w>': 93,
 'highly </w>': 44,
 'transmissible, </w>': 3,
 'infections </w>': 32,
 'caused </w>': 20,
 'by </w>': 692,
 'this </w>': 500,
 'organism </w>': 12,
 'are </w>': 617,
 'relatively </w>': 33,
 'minor </w>': 4,
 'include </w>': 39,
 'pharyngitis, </w>': 2,
 'tracheobronchitis, </w>': 2,
 'bronchiolitis, </w>': 2,
 'croup </w>': 1,
 'with </w>': 1084,
 'fifth </w>': 1,
 'being </w>': 30,
 'asymptomatic. <

In [36]:
from collections import Counter

counts = Counter(text.lower().split())

In [37]:
counts

Counter({'of': 182,
         'the': 138,
         'in': 100,
         'and': 80,
         'to': 63,
         'no•': 45,
         'or': 43,
         'a': 40,
         'is': 35,
         'for': 33,
         'by': 29,
         'with': 28,
         '.': 26,
         'as': 25,
         'that': 25,
         'be': 25,
         'tyrosine': 25,
         'might': 22,
         'nitration': 22,
         'lung': 19,
         'such': 18,
         'inflammatory': 17,
         'not': 17,
         'have': 17,
         'more': 16,
         'formation': 16,
         'respiratory': 15,
         'nos': 15,
         'can': 15,
         'this': 14,
         'are': 13,
         'rns': 13,
         'these': 13,
         'pro-inflammatory': 12,
         'disease': 12,
         'it': 12,
         'protein': 12,
         'its': 11,
         'other': 11,
         'role': 11,
         'nos-2': 10,
         '3-nitrotyrosine': 10,
         'signaling': 10,
         'biological': 9,
         'which': 9,
         'dise

In [29]:
class BytePairEncoding:
    def __init__(self, num_merge):
        assert num_merge != 0
        assert num_merge != None
        
        self.num_merge = num_merge

    def co_occurrence(self, corpus) -> dict:
        pairs = {}
        for word in corpus.split(" "):
            for i in range(len(word) - 1):
                seq = word[i] + " " + word[i + 1]
                if seq in pairs:
                    pairs[seq] += 1
                else:
                    pairs[seq] = 1
        return pairs
        
        

    

In [30]:
encoding = BytePairEncoding(10)

encoding.co_occurrence(text)

{'I n': 15,
 'n t': 131,
 't r': 111,
 'r o': 208,
 'o d': 38,
 'd u': 43,
 'u c': 51,
 'c t': 136,
 't i': 305,
 'i o': 220,
 'o n': 265,
 'n \n': 1,
 'S i': 3,
 'i n': 397,
 'n c': 70,
 'c e': 95,
 'i t': 188,
 't s': 55,
 'd i': 127,
 'i s': 154,
 's c': 12,
 'c o': 78,
 'o v': 14,
 'v e': 125,
 'e r': 191,
 'r y': 74,
 'a s': 133,
 'b i': 25,
 'o l': 70,
 'l o': 37,
 'o g': 32,
 'g i': 22,
 'i c': 114,
 'c a': 85,
 'a l': 149,
 'm e': 87,
 'e s': 217,
 's s': 42,
 's e': 185,
 'e n': 185,
 'n g': 84,
 'g e': 54,
 'm o': 64,
 'l e': 105,
 'e c': 105,
 'c u': 18,
 'u l': 42,
 'o r': 230,
 'r e': 238,
 't h': 310,
 'h a': 105,
 'a n': 216,
 '1 0': 3,
 'y e': 4,
 'e a': 75,
 'a r': 100,
 'r s': 27,
 'a g': 19,
 'g o': 9,
 'o ,': 2,
 'h e': 224,
 'g a': 10,
 'e o': 7,
 'o u': 58,
 'u s': 52,
 'n i': 97,
 'r i': 74,
 'o x': 50,
 'x i': 40,
 'i d': 54,
 'd e': 98,
 '( N': 5,
 'N O': 120,
 'O •': 62,
 '• )': 2,
 'n o': 44,
 'o w': 15,
 'w e': 13,
 'e l': 99,
 'l l': 56,
 'g n': 14,
 'i z':

In [16]:
test = {}
test.update("dog")

ValueError: dictionary update sequence element #0 has length 1; 2 is required